# Patchwork

Patchwork (also known as "Tatami") consists of a square grid divided into regions ("rooms"). Each room must be filled with each of the digits from 1 to the number of cells in the room. Every row and every column must contain the same amount of each digit. Same digits must not be orthogonally adjacent.

------

1. 在网格的每个单元格中输入一个数字，以便大小为 N 的每个区域正好包含一次从 1 到 N 的所有数字。
2. 相同的数字不得正交相邻。
3. 一行或一列中的所有数字必须以相同的频率出现。

In [2]:
def readGrid(path):
    with open(f"../assets/data/patchwork/{path}.txt") as f:
        num = f.readline()
        m, n = int(num.split(" ")[0]), int(num.split(" ")[1])
        grid = []
        cells = []
        for _ in range(m):
            grid.append(f.readline().strip().split(" "))
        f.readline()
        for _ in range(m):
            cells.append(f.readline().strip().split(" "))
                         
        return m, n, grid, cells

if __name__ == "__main__":
    m, n, grid, cells = readGrid("8x8_1")
    for g in grid:
        print(g)
    for c in cells:
        print(c)

['.', '.', '.', '.', '4', '.', '.', '4']
['1', '.', '.', '1', '.', '.', '.', '.']
['.', '.', '4', '.', '2', '.', '.', '.']
['.', '.', '.', '.', '.', '.', '.', '.']
['.', '.', '2', '.', '.', '.', '.', '.']
['.', '.', '.', '.', '.', '.', '.', '.']
['1', '.', '.', '.', '2', '.', '.', '.']
['.', '.', '.', '.', '.', '.', '.', '4']
['A', 'B', 'C', 'C', 'C', 'C', 'D', 'E']
['A', 'B', 'F', 'G', 'H', 'I', 'D', 'E']
['A', 'B', 'F', 'G', 'H', 'I', 'D', 'E']
['A', 'B', 'F', 'G', 'H', 'I', 'D', 'E']
['J', 'K', 'F', 'G', 'H', 'I', 'L', 'M']
['J', 'K', 'N', 'N', 'N', 'N', 'L', 'M']
['J', 'K', 'O', 'O', 'O', 'O', 'L', 'M']
['J', 'K', 'P', 'P', 'P', 'P', 'L', 'M']


In [11]:
from ortools.sat.python import cp_model as cp
class PatchworkSolver:
    
    def __init__(self, grid, cells, X, Y):
        self.grid , self.cells, self.X, self.Y = grid, cells, X, Y 
        self.x = {}
        self.model = cp.CpModel()
        self.solver = cp.CpSolver()
        self.cell_dict = dict()
    
    def solve(self):
        self.cell_size = 0
        for i in range(self.X):
            for j in range(self.Y):
                if self.cells[i][j] not in self.cell_dict:
                    self.cell_dict[self.cells[i][j]] = [(i, j)]
                else:
                    self.cell_dict[self.cells[i][j]].append((i, j))
        for i in range(self.X):
            for j in range(self.Y):
                self.cell_size = max(self.cell_size, len(self.cell_dict[self.cells[i][j]]))
                for k in range(len(self.cell_dict[self.cells[i][j]])):
                    self.x[i, j, k + 1] = self.model.NewBoolVar(f"x[{i}_{j}_{k + 1}]")
        
        for i in range(self.X):
            for j in range(self.Y):
                self.model.Add(sum(self.x[i, j , k] for k in range(1, self.cell_size + 1)) == 1)
                if self.grid[i][j] != ".":
                    self.model.Add(self.x[i, j , int(self.grid[i][j])] == 1)
        
        for k, v in self.cell_dict.items():
            for target in range(1, len(v) + 1): 
                self.model.Add(sum([self.x[subx, suby, target ] for (subx, suby) in v]) == 1)
        
        for i in range(self.X):
            for target in range(1, self.cell_size ):
                self.model.Add(sum([ self.x[i, j, target] for j in range(self.Y)]) == sum([ self.x[i, j, target + 1] for j in range(self.Y)]))

        for j in range(self.Y):
            for target in range(1, self.cell_size ):
                self.model.Add(sum([ self.x[i, j, target] for i in range(self.X)]) == sum([ self.x[i, j, target + 1] for i in range(self.X)]))
        
        for i in range(self.X):
            for j in range(self.Y - 1):
                for k in range(1, self.cell_size + 1):
                    self.model.Add(self.x[i, j, k] + self.x[i, j + 1, k ] <= 1)
        
        for j in range(self.Y):
            for i in range(self.X - 1):
                for k in range(1, self.cell_size + 1):
                    self.model.Add(self.x[i, j, k] + self.x[i + 1, j, k ] <= 1)
        status = self.solver.Solve(self.model)
        
        if status == cp.OPTIMAL:
            for i in range(self.X):
                for j in range(self.Y):
                    for target in range(1, self.cell_size + 1):
                        if self.solver.Value(self.x[i, j, target]) > 0.5:
                            print(target, end = " ")
                print()
            print()
            
if __name__ == "__main__":
    m, n, grid, cells = readGrid("8x8_1")
    PatchworkSolverTest = PatchworkSolver(grid, cells, m, n)
    PatchworkSolverTest.solve()
                

3 2 1 2 4 3 1 4 
1 4 3 1 3 2 4 2 
2 1 4 3 2 4 3 1 
4 3 1 2 4 1 2 3 
3 1 2 4 1 3 4 2 
4 2 4 1 3 2 3 1 
1 4 3 4 2 1 2 3 
2 3 2 3 1 4 1 4 

